## Imports

In [ ]:
import requests
import urllib.request
from markdownify import markdownify as md
from bs4 import BeautifulSoup
from dotenv import dotenv_values
from openai import AzureOpenAI
import json

config = dotenv_values()

client = AzureOpenAI(
    api_key=config["AZURE_OPENAI_API_KEY"],
    azure_endpoint=config["AZURE_OPENAI_API_BASE"],
    api_version=config["AZURE_OPENAI_API_VERSION"]
)
openai_chatmodel = config["AZURE_OPENAI_CHAT_MODEL"]

GRAY = "\033[90m"
BOLD = "\033[1m"
RESET = "\033[0m"

## Setup

In [ ]:
def get_search_results_for(query):   
    encoded_query = urllib.parse.urlencode({'q': query})
    url = f'https://html.duckduckgo.com/html?q={encoded_query}'

    request = urllib.request.Request(url)
    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

    raw_response = urllib.request.urlopen(request).read()
    html = raw_response.decode("utf-8")

    soup = BeautifulSoup(html, 'html.parser')
    a_results = soup.select("a.result__a")

    links = []
    for a_result in a_results:
        # print(a_result)
        url = a_result.attrs['href']
        title = a_result.text
        links.append({"title": title,  "url": url} )
        
    return links


def load_page_content(url) -> str:
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'})
    page_content = response.content.decode('utf-8')
    page_content_md = md(page_content)
    
    return page_content_md

def get_homepage(blogger: str) -> str:
    # return "https://vladiliescu.net"
    return get_search_results_for(f"{blogger} homepage")[0]["url"]



In [ ]:
query = "Tell me what Vlad Iliescu's latest blog post is about"

### Single LLM Call

In [ ]:
completion = client.chat.completions.create(
    model=openai_chatmodel,
    messages=[{"role": "user", "content": query}],
    temperature=0.0
)

print(completion.choices[0].message.content)

### Tool Calling

In [ ]:
tools = [{
        "type": "function",
        "function": {
            "name": "get_homepage",
            "description": "Returns the homepage of a particular blogger.",
            "parameters": {
                "type": "object",
                "properties": {
                    "blogger": {
                        "type": "string",
                        "description": "Blogger name"
                    }
                },
                "required": [
                    "blogger"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "load_page_content",
            "description": "Returns the content of a particular webpage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Url of the webpage for which to retrieve the content"
                    }
                },
                "required": [
                    "url"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

def call_function(name, args):
    if name == "get_homepage":
        return get_homepage(**args)
    if name == "load_page_content":
        return load_page_content(**args)    


In [ ]:

messages = [{"role": "user", "content": query}]

total_input_token_count = 0
total_output_token_count = 0

while (True):
    completion = client.chat.completions.create(
        model=openai_chatmodel,
        messages=messages,
        tools=tools
    )

    total_input_token_count += completion.usage.prompt_tokens
    total_output_token_count += completion.usage.completion_tokens

    if completion.choices[0].finish_reason == "stop":
        print(f"{BOLD}Final answer: {completion.choices[0].message.content}{RESET}")
        break
    elif completion.choices[0].finish_reason == "tool_calls":
        messages.append(completion.choices[0].message)
        for tool_call in completion.choices[0].message.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            result = call_function(name, args)
            print(f"Called {BOLD}{name}({args}){RESET} and it returned {GRAY}{result[:300]}{RESET}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })
    else:
        raise Exception("We're not supposed to be here")

In [ ]:
# GPT-4o-2024-1120 Pricing (Sweden Central)

input_cost=2.90308/1_000_000
output_cost=11.6123/1_000_000

total_cost = total_input_token_count * input_cost + \
            total_output_token_count * output_cost


print(f"Cost: €{total_cost:.4f} (Input Tokens: {total_input_token_count:,}, Output Tokens: {total_output_token_count:,})")